In [778]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [779]:
product_type = "ambient"
num_time_periods = 7

In [780]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [781]:

if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 300
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
max_order_exceed_multiplier = 1.5

In [782]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [783]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [784]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [785]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [786]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [787]:
def getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - cum_demand > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - cum_demand) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost 
    return max_order 

In [788]:
def silverMeal(item):
    
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0

    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue
            cum_demand += item_demand[t]
            
            if(t==order_date+life):
                orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) 
                tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item) 
            if(math.ceil(cum_demand/cu_per_tu_item) > maximum_order_quantity_tu_item):
                penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier * variable_cost
                #penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier  
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    
                    orders[order_date] = math.ceil(cum_demand/cu_per_tu_item)

                    #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item,variable_cost, maximum_order_quantity_tu_item)
                    tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
                    tot_ordering += math.ceil(cum_demand/cu_per_tu_item) * variable_cost
                    tot_fixed += fixed_cost   
                    tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                 
                    run_complete = True
                    break

            else:
                orders[order_date] = math.ceil((cum_demand- item_demand[t])/cu_per_tu_item)

                #cost+= getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost) + getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
                tot_disposal += getCost(orders, order_date, cu_per_tu_item, cum_demand- item_demand[t], disposal_cost)
                tot_ordering += math.ceil((cum_demand- item_demand[t])/cu_per_tu_item) * variable_cost
                tot_fixed += fixed_cost
                tot_max_order +=  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)

                order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
        
    
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


857291.4500000019
Ordering 708690.4999999983
Fixing  106309.69999999995
Clearance  33595.899999999965
Max Order  8695.35


In [789]:
schedule

{'00027afc': {0: 3, 4: 2},
 '00084396': {0: 2},
 '0025253f': {0: 4, 2: 6},
 '00299fa6': {0: 1, 3: 2},
 '002b8d7d': {0: 2},
 '0034e982': {0: 3, 2: 4},
 '00415c43': {0: 14, 4: 8},
 '0057e9b7': {0: 5, 2: 19},
 '0059a703': {0: 12, 4: 5},
 '0062b624': {0: 5},
 '00718852': {0: 12},
 '007d0c15': {0: 1, 4: 1},
 '009aa26d': {0: 8},
 '00d09a16': {0: 2, 4: 1},
 '00f7bc48': {0: 1, 4: 1},
 '00f95978': {0: 18},
 '010001ce': {0: 2, 5: 1},
 '01039c78': {0: 5, 4: 4},
 '010a9e81': {0: 1},
 '011f544e': {0: 1, 4: 1},
 '01527ae0': {0: 1},
 '01581e6a': {0: 46},
 '015bf73f': {0: 2, 3: 2},
 '016d54fd': {0: 1, 5: 1},
 '0173b075': {0: 2, 4: 2},
 '018a81b7': {0: 4, 5: 1},
 '018afad9': {0: 4, 3: 5},
 '019b9022': {0: 1},
 '01c1f8cb': {0: 2},
 '01c4e30e': {0: 1, 3: 1},
 '01c5bf11': {0: 1, 4: 1},
 '01cac4fc': {0: 1},
 '01cb9c42': {0: 4, 2: 8, 4: 9},
 '01ea7fcd': {0: 8},
 '01ed569a': {0: 20},
 '01f22aa0': {0: 1, 3: 1},
 '01f433bd': {0: 1, 3: 2},
 '02031c33': {0: 94},
 '0220bc95': {0: 1},
 '022168b8': {0: 2, 3: 1, 5: 

In [790]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol
    if vol > warehouse_volume:

        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost
obj_val = obj_val + buf_cost

In [791]:
obj_val

857291.4500000019

In [792]:
volumes

{0: 428.72424740500026,
 1: 352.1333585470004,
 2: 296.8650959119995,
 3: 306.380654603998,
 4: 253.641881803997,
 5: 193.81620701699728}

In [793]:
max_improvement = math.inf
cost = obj_val
modified_schedule = {}
day_wise_orders = {}

buffer_penalty_tot = 0
max_order_penalty_tot = 0
fixed_ordering_cost_tot = 0 

count = 0
iterator = 0
MAX_ITER = 100
MAX_REPEAT = 3
prev_max_improvement = 0

while count < MAX_REPEAT and iterator < MAX_ITER:
    for item in schedule:
        buffer_penalty = 0
        max_order_penalty = 0 
        cost_improvement = 0
        modified_orders = {}
        # Check if the item has a second order date
        if len(schedule[item]) > 1:
            order_days = list(schedule[item].keys())
            for i in range (0,len(order_days)-1, 2):
                day1 = i
                day2 = i +1
                    
                if order_days[day2] - order_days[day1] <= shelf_life[item]:
                    tot_orders = schedule[item][order_days[day1]]+ schedule[item][order_days[day2]]
                    vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                    if vol + volumes[day1] > warehouse_volume:
                        buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
                    if tot_orders > maximum_order_quantity_tu[item]:
                        max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                    delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                    if delta < 0:
                        fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                        buffer_penalty_tot += buffer_penalty
                        max_order_penalty_tot += max_order_penalty
                        cost += delta
                        modified_orders[day1] = tot_orders
                        volumes[day1] += vol
                    else:
                        modified_orders[day1] = schedule[item][order_days[day1]]
                        modified_orders[day2] = schedule[item][order_days[day2]]
            modified_schedule[item] = modified_orders
        
        else:
            if len(schedule[item]) == 1 : 
                modified_schedule[item] = schedule[item]
            else:
                continue


        if cost!=0 and cost < max_improvement:
            max_improvement = cost
    
    print("Buffer penalty paid", buffer_penalty_tot)
    print("max order penalty paid", max_order_penalty_tot)
    print("fixed cost reduced by", fixed_ordering_cost_tot)
    schedule = modified_schedule
    obj_val = max_improvement
    iterator +=1

    print(max_improvement)

    if(max_improvement==prev_max_improvement):
        count+=1
    prev_max_improvement = max_improvement

Buffer penalty paid 169.74489900002823
max order penalty paid 6.6
fixed cost reduced by 24316.100000000013
833151.6948990037
Buffer penalty paid 169.74489900002823
max order penalty paid 6.6
fixed cost reduced by 24316.100000000013
833151.6948990037
Buffer penalty paid 169.74489900002823
max order penalty paid 6.6
fixed cost reduced by 24316.100000000013
833151.6948990037
Buffer penalty paid 169.74489900002823
max order penalty paid 6.6
fixed cost reduced by 24316.100000000013
833151.6948990037
